<img src="https://raw.githubusercontent.com/UXARRAY/uxarray/main/docs/_static/images/logos/uxarray_logo_h_dark.svg"
     width="30%"
     alt="UXarray logo"
     align="right"
/>

# E3SM Atmosphere 

This recipe demonstrates how to analyze unstructured grid output from the Energy Exascale Earth System Model (E3SM) using UXarray. Unlike traditional approaches that require regridding, UXarray enables direct analysis of the native model output, preserving the full fidelity of the simulation data.

## Objectives

This recipe guides you through calculating and visualizing key atmospheric radiation metrics:

1. Shortwave Cloud Radiative Effect (SWCRE)
2. Longwave Cloud Radiative Effect (LWCRE)
3. Net Cloud Radiative Effect (NetCRE)

The workflow includes techniques for:

- Computing radiation components directly on the native unstructured grid
- Visualizing spatial patterns between two time steps
- Analyzing temporal evolution through difference plots


-----

In [ ]:
import cartopy.crs as ccrs
import holoviews as hv
import uxarray as ux

hv.extension("bokeh")

## Data

The example uses output from an E3SMv2 atmosphere-only (AMIP) simulation with the following specifications:

- Simulation Period: 6 years
- Configuration: Present-day control forcing (F2010)
- Resolution: 1-degree horizontal (ne30pg2)
- Boundary Conditions: Default E3SMv2 settings for sea surface temperatures and sea ice, cycling annually

In [ ]:
grid_path = "../../meshfiles/ne30pg2.grid.nc"
data_path = "../../meshfiles/ne30pg2.data.nc"

uxds = ux.open_dataset(grid_path, data_path)

## Calculating Shortwave Cloud Radiative Effect (SWCRE)

Shortwave cloud radiative effect can be approximated as the difference beteween all-sky net shortwave flux (FSNT) at the top of the model and the clear-sky net shortwave flux (FSNTC).

$$SWCRE = FSNT - FSNTC$$



In [ ]:
uxds["SWCRE"] = uxds["FSNT"] - uxds["FSNTC"]
uxds["SWCRE"]

## Calculating Longwave Cloud Radiative Effect (LWCRE)

Longwave cloud radiative effect is similar to that for SWCRE, but the all-sky and clear-sky longwave fluxes are applied instead.

$$LWCRE = FLUT - FLUTC$$

In [ ]:
uxds["LWCRE"] = uxds["FLUT"] - uxds["FLUTC"]
uxds["LWCRE"]


## Calculating Net Cloud Radiative Effect (NetCRE)

Net cloud radiative effect is thus the difference beteween shortwave and longwave cloud radiative effect.

$$netCRE = SWCRE - LWCRE$$

In [ ]:
uxds["netCRE"] = uxds["SWCRE"] - uxds["LWCRE"]
uxds["netCRE"]

## Plotting

Having computed the Net Cloud Radiative Effect (NetCRE), we can now create visualizations to examine the spatial patterns and temporal evolution of cloud-radiation interactions. Our visualization approach focuses on three key aspects:

1. Initial conditions represented by the first timestep
2. Final conditions from the last timestep 
3. The difference between these states to reveal temporal changes

The following sections demonstrate how to create these visualizations while maintaining the native unstructured grid structure, ensuring we preserve the full resolution of our simulation data. Below we declare our desired projection.


In [ ]:
projection = ccrs.Robinson()

### Visualizing Different Time Steps

The following visualization displays the Net Cloud Radiative Effect at two critical points in our model output: the first January (initial state) and the final month of January. This comparison allows us to examine both the baseline conditions and the evolved state of cloud-radiation interactions.


In [ ]:
(
    uxds["netCRE"]
    .isel(time=0)
    .plot(
        projection=projection,
        pixel_ratio=4.0,
        coastline=True,
        title="First Time Step (Year 0 Jan)",
    )
    + uxds["netCRE"]
    .isel(time=61)
    .plot(
        projection=projection,
        pixel_ratio=4.0,
        coastline=True,
        title="Final Time Step (Year 6 Jan)",
    )
).cols(1)

### Difference Analysis

To quantify and visualize how the Net Cloud Radiative Effect evolved over the simulation period, we compute the difference between the final and initial months of January. This differential analysis highlights regions where cloud-radiation interactions have strengthened or weakened during the simulation.


In [ ]:
(uxds["netCRE"].isel(time=61) - uxds["netCRE"].isel(time=-0)).plot(
    projection=projection, coastline=True, pixel_ratio=4.0, title="Change in NetCRE"
)